# Are fantasy football news reports reliable indicators of player workload?

By Vinesh Kannan | January 2026

In [1]:
import duckdb
import json

In [2]:
DATABASE = "../database.db"

In [3]:
def show_report(path: str) -> str:
    with open(path, "r") as file:
        print(json.dumps(json.loads(file.read()), indent=2))

# Introduction

When Carolina Panthers running back Chuba Hubbard went down with an injury in week 4 of the 2025 NFL season, did you immediately pick up Rico Dowdle?

Would this news report have enticed you?

In [4]:
show_report(path="../docs/assets/reports/rotoballer_203375.json")

{
  "title": "Rico Dowdle in Line For an Increased Workload in Week 5",
  "description": "Carolina Panthers running back and former Dallas Cowboy, Rico Dowdle, is set to be the lead back in the Panthers' backfield in Week 5 against the Miami Dolphins, with Chuba Hubbard (calf) ruled out for Week 5. Hubbard is considered week-to-week, so Dowdle isn't expected to be a hot commodity for a season-long play, but he immediately rises in the Week 5 ranks and becomes a top-25 running back against the Dolphins defense, which allows the seventh most fantasy points to the running back position. Dowdle is no slouch either; last year with the Cowboys, he accumulated 1,328 all-purpose yards and five touchdowns. Rookie running back Trevor Etienne becomes the team's RB2 and should also be in line for additional carries.",
  "player_name": "Rico Dowdle",
  "week": 5,
  "team": "CAR",
  "opp": "MIA",
  "att": 23,
  "tgt": 4,
  "touches": 27,
  "actual": "high",
  "expect": "high",
  "reason": "injury",


<!--
We should create a common component for displaying news reports and the surrounding context.

Use that component to display the JSON above.
-->

If you did snag and start Rico Dowdle in week 5, you would have played the number one running back.

This report provides seemingly reliable intelligence:

1. Chuba Hubbard is injured
2. Rico Dowdle will take his workload
3. Their next opponent, the Dolphins, are weak against the run

Points 1 and 2 are clear enough for a reader to expect that Rico Dowdle will get more touches in week 5. Whether he will produce scoring value from those touches is harder to say. Point 3 is a bit more open-ended: maybe the Dolphins will have a better matchup against the Panthers or maybe Rico Dowdle will struggle in a lead role.

Fantasy performance can be unpredictable. But maybe player workload is more predictable.

There are endless articles, podcasts, and commentary about the NFL and fantasy football that could provide insight or gossip to inform your fantasy lineup. But these fantasy news reports are a bit more concise and standardized.

When these kinds of reports suggest that a player will get the ball more next week, how often does that actually happen?

# Methodology

To answer this question, I analyzed 2,000 fantasy news reports about running backs from the 2025 season.

The process boils down to this:

1. Assign reports to upcoming games
2. Classify expected workload from reports
3. Compare against actual workload

## Data Collection

I sourced the player statistics and news reports from Sleeper and RotoBaller. The criteria for inclusion were:

1. Player is an active running back or full back (can always add WRs and TEs later)
2. Player only played on one team during the season (simplifies the process of matching news reports to games, only drops a few players)
3. Report is after week one (too much volume and noise in reports before the season starts)
4. Report is prospective, discussing expectations for next week, not just a recap of the previous week
5. Player actually played in the game the week of the report
6. Player also played in the game the week before the report

Luckily, criteria 4 was easy to enforce because Sleeper and RotoBaller separate game recaps as "analysis" news. Choosing only the non-analysis reports led to the kind of reports aimed at fantasy managers trying to decide who to start and who to sit.

This whittled us down from over 5,000 potential reports to 2,040 reports. The final dataset covers 117 players.

## Report Classification

I randomly sampled 36 reports from that set and manually labeled them.

Each report is classified into both an expected workload level and a reason category.

Expected workload level:

- High: more than 20 touches (carries + targets), usually the lead back
- Medium: 10-20 touches, usually a split backfield
- Low: less than 10 touches
- Unknown: cannot be determined from the report

Reason category:

- Performance: the player is playing well, taking touches from a teammate who is playing worse, etc.
- Injury: the player is injured, back from injury, their teammate was injured, etc.
- Opponent: the team has a good or bad match up next week
- Unknown: none of the above or cannot be determined

Manually classifying the reports made me optimistic that a large language model could do the same. I used three of the manually-labeled reports as examples in the model prompt and the remaining 33 as a validation set to score different models. More details on this in the appendix.

## Comparison Metrics

Finally, I compared the expected workload level extracted from the report to the player's actual number of touches (carries + targets) in their next game.

Primarily, I used two metrics to analyze how well these report predict actual workload:

1. Precision: when the report suggests a certain workload level, how often is that workload level what actually happens?
2. Recall: out of all the players who actually had a certain workload level, how many were expected by reports?

As you can imagine, there are far more players with low or medium workload than high workload. So it will be easiest to interpret the precision and recall scores if we break them out by each workload level.

# Results

## Overall Results

Spoiler alert: fantasy news reports are not particularly predictive of actual workload.

In [5]:
scores_overall = {}
with duckdb.connect(DATABASE) as con:
    cur = con.sql("""
    SELECT
        count(1) AS reports,
        count_if(expect = 'low') AS expected_low,
        count_if(expect = 'medium') AS expected_medium,
        count_if(expect = 'high') AS expected_high,
        count_if(actual = 'low') AS actually_low,
        count_if(actual = 'medium') AS actually_medium,
        count_if(actual = 'high') AS actually_high,
        sum(tp_lo) / sum(tp_lo + fp_lo) AS overall_precision_low,
        sum(tp_md) / sum(tp_md + fp_md) AS overall_precision_medium,
        sum(tp_hi) / sum(tp_hi + fp_hi) AS overall_precision_high,
        sum(tp_lo) / sum(tp_lo + fn_lo) AS overall_recall_low,
        sum(tp_md) / sum(tp_md + fn_md) AS overall_recall_medium,
        sum(tp_hi) / sum(tp_hi + fn_hi) AS overall_recall_high,
    FROM prediction_comparison
    ;
    """)
    scores_overall = cur.df().to_dict(orient="records")

with open("../docs/assets/results/scores_overall.json", "w") as outfile:
    outfile.write(json.dumps(scores_overall))

print(json.dumps(scores_overall, indent=2))

[
  {
    "reports": 2040,
    "expected_low": 768.0,
    "expected_medium": 569.0,
    "expected_high": 641.0,
    "actually_low": 830.0,
    "actually_medium": 808.0,
    "actually_high": 402.0,
    "overall_precision_low": 0.6614583333333334,
    "overall_precision_medium": 0.46397188049209137,
    "overall_precision_high": 0.4040561622464899,
    "overall_recall_low": 0.6120481927710844,
    "overall_recall_medium": 0.32673267326732675,
    "overall_recall_high": 0.6442786069651741
  }
]


<!--
Display the results from the JSON above as a table like this:

- The columns should be the workload level
- The rows are:
  - The number of reports that expected that level
    - You can get this from the JSON object using keys like "expected_X" for each level
    - Also show in parentheses the percentage that number represents out of all reports
    - You can calculate this by summing up the three "expected_X" level values to get the denominator
  - The number of actual workloads at that level
    - You can get this from the JSON object using keys like "actually_X" for each level
    - Similar to above, also show the percentage that number represents
  - The precision at that level
    - You can get this from the JSON object using keys like "overall_precision_X" for each level
  - The recall at that level
    - You can get this from the JSON object using keys like "overall_recall_X" for each level

|-----------------------|--------|--------|--------|
|                       | Workload Level           |
|-----------------------|--------|--------|--------|
|                       | Low    | Medium | High   |
|-----------------------|--------|--------|--------|
| Reported Expectations | N (%)  | N (%)  | N (%)  |
| Actual Workloads      | N (%)  | N (%)  | N (%)  |
| Precision             | %      | %      | %      |
| Recall                | %      | %      | %      |
|-----------------------|--------|--------|--------|

-->

Fantasy reports seem to trend optimistic: the reported expectations are higher than the actual workloads.

Since there are more actual cases of low workloads than medium or high, that also makes it easier to predict low workloads.

- 66.1% of reports that expect low workload are actually correct (precision)
- 61.2% of all low workloads were correctly expected by reports (recall)

Of course, fantasy managers want to know about high workloads.

- Only 40.4% of reports that expect high workload are actually correct (precision)
- While 64.4% of all high workloads were correctly expected by reports (recall)

Having good recall but low precision indicates excessive optimism.

In [6]:
show_report(path="../docs/assets/reports/rotoballer_203459.json")

{
  "title": "Saquon Barkley Looks to Overcome Denver Defense",
  "description": "Philadelphia Eagles running back Saquon Barkley has failed to eclipse 50 rushing yards in each of his last two games, and will face another hurdle this Sunday against the Broncos. Denver is allowing 99.3 rushing yards per game this season-just one of 11 teams to average under 100. They also stifled Bengals RB Chase Brown on Monday, keeping him to 40 yards on 10 carries with his longest attempt going for six yards. However, Philly does have a much stronger offensive line than Cincinnati does, and Barkley averages more yards per carry (3.1) than Brown (2.3). Even with a couple things going against him, Barkley remains a definitive RB1 for Week 5.",
  "player_name": "Saquon Barkley",
  "week": 5,
  "team": "PHI",
  "opp": "DEN",
  "att": 6,
  "tgt": 3,
  "touches": 9,
  "actual": "low",
  "expect": "high",
  "reason": "opponent",
  "rushing_touchdowns": 0,
  "receiving_touchdowns": 1,
  "ppr_scoring_fantasy_

This report about Saquon Barkley suggested that he would be an RB1 in week 5 against the Broncos. But in the end, he only got six carries and three targets. Who could have expected that [the Eagles would run the ball just **once** in the second half](https://www.nbcsportsphiladelphia.com/nfl/philadelphia-eagles/saquon-barkley-eagles-vs-broncos-eagles-offense-nfl-week-5/688174/) while protecting a 14-point lead?

Reports want to encourage us that a player will get a lot of touches, so they spread a wide net and capture the 64% of cases where the workload actually was high. Another way to look at that recall score is that roughly 36% of actually high workloads were unexpected based on reporting: the breakout workhorses that the news did not expect.

The medium workloads are even harder to predict, with both sub-50% precision and recall.

For the cases where the reports are wrong, which direction are they wrong?

To understand this, we look at a confusion matrix: the rows represent the actual workload and the columns represent the expected workload.

![Confusion Matrix: 2025 Season](../docs/assets/results/classification_season_2025.png)

The corners show the fringes:

- Breakout stars: 33 reports predicted a low workload, but the player actually had a high workload
- Underperformers: 86 reports predicted a high workload, but the player actually had a low workload
- Unclear: 62 reports were not clear on what level of workload they expected (summed up in the unknown column)

Here is a case where our model could not figure out the expected workload level from a report:

In [7]:
show_report(path="../docs/assets/reports/rotoballer_207371.json")

{
  "title": "Sloppy Offense Leads To Reduced Touches For Aaron Jones Sr., Jordan Mason",
  "description": "Minnesota Vikings running backs Aaron Jones Sr. and Jordan Mason combined for only 13 carries in the team's Week 10 loss to Baltimore, but the running game was hindered by a significant number of presnap penalties. Minnesota committed a whopping eight false start penalties -- the most by a team since 2011 -- including five of them on first-and-10 situations, and also committed three turnovers. Both Jones and Mason were above five years per carry, but Minnesota found itself in so many poor down-and-distance situations that it attempted 42 passes. \"When you average six yards a play on offense, it's all for naught if you're going to be giving back so many of those yards in different capacities,\" Vikings head coach Kevin O'Connell said. It was a frustrating day for managers who started Jones or Mason, but an aberrational amount of penalties suggests Minnesota's top two running back

The report provides some objective data (only 13 carries between the two backs) but ultimately pins the blame on penalties. This makes it unclear what the report expects Jones' workload to be next week.

Whether the Vikings cleaned up their act, or whether O'Connell decided to feed Jones the rock, or whether the Bears defense sh*t the bed, Aaron Jones got a high workload in week 11: 16 rushes and six targets. Meanwhile, Jordan Mason only got six carries and wasn't involved in the passing game at all. Who could have guessed?

## Expected Workload Reasons

Are certain types of reports more predictive of workload?

Once again, the trend is that lower workload, which is more common, is easier to predict. Especially when it's tied to an injury report.

In reality, a fantasy report will base its recommendation on multiple reasons, but I urged the model to narrow it down to the dominant reason, if any.

In [8]:
scores_by_reason = []
with duckdb.connect(DATABASE) as con:
    cur = con.sql("""
    SELECT
        reason,
        count(1) AS reports,
        sum(tp_lo) / sum(tp_lo + fp_lo) AS precision_low,
        sum(tp_md) / sum(tp_md + fp_md) AS precision_medium,
        sum(tp_hi) / sum(tp_hi + fp_hi) AS precision_high,
        sum(tp_lo) / sum(tp_lo + fn_lo) AS recall_low,
        sum(tp_md) / sum(tp_md + fn_md) AS recall_medium,
        sum(tp_hi) / sum(tp_hi + fn_hi) AS recall_high,
    FROM prediction_comparison
    GROUP BY reason
    ORDER BY reports DESC
    ;
    """)
    scores_by_reason = cur.df().to_dict(orient="records")

with open("../docs/assets/results/scores_by_reason.json", "w") as outfile:
    outfile.write(json.dumps(scores_by_reason))

print(json.dumps(scores_by_reason, indent=2))

[
  {
    "reason": "injury",
    "reports": 870,
    "precision_low": 0.7017114914425427,
    "precision_medium": 0.3881278538812785,
    "precision_high": 0.29577464788732394,
    "recall_low": 0.6266375545851528,
    "recall_medium": 0.29310344827586204,
    "recall_high": 0.5163934426229508
  },
  {
    "reason": "performance",
    "reports": 802,
    "precision_low": 0.6791666666666667,
    "precision_medium": 0.49130434782608695,
    "precision_high": 0.4773413897280967,
    "recall_low": 0.6127819548872181,
    "recall_medium": 0.3373134328358209,
    "recall_high": 0.7860696517412935
  },
  {
    "reason": "opponent",
    "reports": 298,
    "precision_low": 0.46,
    "precision_medium": 0.5769230769230769,
    "precision_high": 0.39361702127659576,
    "recall_low": 0.6301369863013698,
    "recall_medium": 0.379746835443038,
    "recall_high": 0.5522388059701493
  },
  {
    "reason": "unknown",
    "reports": 70,
    "precision_low": 0.631578947368421,
    "precision_medium":

<!--
Display the results from the JSON above as a table like this:

- The columns should be the workload level
- The rows should be the reason categories
- The values are:
  - The number of reports for each reason category
    - You can get this from the JSON data using the "reports" key for each entry
    - Also show in parentheses the percentage that number represents out of all reports
    - You can calculate this by summing up all the "reports" keys across all entries
  - The precision at each workload level
    - You can get this from the JSON data using the "precision_X" key for each entry
  - The recall at each workload level
    - You can get this from the JSON data using the "recall_X" key for each entry

|-------------|---------|--------|--------|--------|--------|--------|--------|
|                       | Precision                | Recall                   |
|-------------|---------|--------|--------|--------|--------|--------|--------|
| Reason      | Reports | Low    | Medium | High   | Low    | Medium | High   |
|-------------|---------|--------|--------|--------|--------|--------|--------|
| Injury      | N (%)   | %      | %      | %      | %      | %      | %      |
| Performance | N (%)   | %      | %      | %      | %      | %      | %      |
| Opponent    | N (%)   | %      | %      | %      | %      | %      | %      |
| Unknown     | N (%)   | %      | %      | %      | %      | %      | %      |
|-------------|---------|--------|--------|--------|--------|--------|--------|

-->

Injury is the biggest theme amongst the reports, but it can reflect in multiple ways:

- Fairly obvious: A player is injured, so their workload will be low or none, this is correct 70.1% of the time (precision)
- More speculative: Another player is injured, so this player will get more workload, correct 38% of the time at the medium workload level and 29% at the high workload level (precision)

Rico Dowdle's meteoric rise to power in week 5 is a great example of a breakout performance that could have been predicted by an injury to the lead back.

Here is another interesting example where an injury report tempers expectations for a lead back:

In [9]:
show_report(path="../docs/assets/reports/rotoballer_207739.json")

{
  "title": "Bam Knight Considered Questionable for Week 11",
  "description": "Arizona Cardinals running back Bam Knight (ankle) was limited in practice all week and is listed as questionable to play on Sunday against the division-rival San Francisco 49ers in Week 11. Knight was once again the team's starting RB in last weekend's blowout loss to the Seattle Seahawks, with Trey Benson (knee) still on Injured Reserve, but now his status is up in the air. The 24-year-old should be able to suit up, but his ankle injury could open the door for both Emari Demercado and Michael Carter to see more backfield touches. Knight had only 33 scrimmage yards on 11 touches before his ankle injury in Week 10, and he hasn't been very efficient since taking over in Arizona. Even when healthy, Knight's RB1 role hasn't been very secure, with Demercado seeing 14 carries in Week 9. If Knight is active, he'll be a pretty wishy-washy RB3/flex with limited upside for fantasy managers.",
  "player_name": "Zonov

RotoBaller warned us that even though Zonovan Knight might play in week 9 against the 49ers, his workload could be lower. Knight's fantasy value was salvaged by a rushing touchdown, but the report correctly predicted that Knight would not eclipse 10 touches, getting just five carries and four targets.

Performance is a more nebulous category. It makes sense that low performance predicts low workload, but we also get closer to 50% on expecting medium or high workload based on performance. Recall is high here, with 78.6% of actual high workloads having been predicted by a news report. This could be because the top running backs on each team are likely to reliably get a high workload each week.

Reports based on opponents can also go either way. If a report says a player will get low workload, maybe due to a tough opponent, that only actually happened in 46% of cases (precision). If you read a report saying that a player could exploit a weak opponent to get a high workload, that was only correct in 39% of cases (precision).

Here is an example of a report classified as opponent-based that also discusses performance, but ultimately takes a pessimistic outlook.

In [10]:
show_report(path="../docs/assets/reports/rotoballer_206534.json")

{
  "title": "Ashton Jeanty Salvages his Day With Receiving Touchdown in Week 9",
  "description": "Las Vegas Raiders rookie first-round running back Ashton Jeanty had another mediocre performance on the ground in the team's Week 9 overtime loss to the visiting Jacksonville Jaguars on Sunday, but he salvaged his day for fantasy managers by catching all five of his targets for 47 yards and a touchdown through the air. On the ground, Jeanty handled a team-high 13 carries for 42 yards (3.2 yards per carry). The 21-year-old was able to bounce back coming out of the bye week after he had a season-low six carries for 21 yards and only one catch for 13 yards in the Week 7 blowout loss to the Kansas City Chiefs. It's been slow going for Jeanty the last three games behind a shaky Raiders offensive line, and things won't get any easier on a short week on Thursday Night Football against a stout Denver Broncos defense in Week 10. Through his first eight NFL games, Jeanty has three rushing scores a

Ultimately, Jeanty got a high workload with 19 carries and 5 target, putting up a respectable 15 points in PPR leagues. You could be forgiven for not acting on this report.

## Is Timing Everything?

Do report expectations get more predictive of workload as the season goes on?

TODO

Are reports on game day more predictive of workload?

TODO

## All Reports

TODO